<h1>IMPORT NECESSARY LIBRARIES</h1>

In [ ]:
import os
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib as plt



<h1>SOURCES FOR DATASET</H1>
<h3>https://www.kaggle.com/datasets/akshat103/e-waste-image-dataset</h3>
<h3>https://www.kaggle.com/datasets/mostafaabla/garbage-classification</h3>
<h3>https://www.kaggle.com/datasets/vencerlanz09/plastic-paper-garbage-bag-synthetic-images</h3>
The dataset used for this project is a collection from various sources

class_labels = {
    0: 'Biological',
    1: 'Cardboard',
    2: 'Clothes',
    3: 'E-waste',
    4: 'Glass',
    5: 'Metal',
    6: 'Paper',
    7: 'Plastic',
    8: 'Shoes'
}

<h1>UPDATE THE PARAMETERS AS REQUIRED</h1>

In [ ]:
IMAGE_SIZE = (150, 150)
BATCH_SIZE = 32
NUM_CLASSES = 9  # Update this to match the number of classes in your dataset
EPOCHS = 10


<h1>class_distribution_of_dataset</h1>

In [ ]:


# Directory containing your dataset
dataset_dir = r"directory_of_dataset"

# Get the class names (subdirectories in the dataset directory)
class_names = os.listdir(dataset_dir)

# Count the number of items in each class
class_counts = {}
for class_name in class_names:
    class_dir = os.path.join(dataset_dir, class_name)
    num_items = len(os.listdir(class_dir))
    class_counts[class_name] = num_items

# Plot the bar graph
plt.figure(figsize=(10, 6))
plt.bar(class_counts.keys(), class_counts.values(), color='skyblue')
plt.xlabel('Classes')
plt.ylabel('Number of Items')
plt.title('Number of Items in Each Class')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better visibility
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()

In [ ]:
train_dir = r"E:\DATASETS"
validation_dir = r"E:\validation"


<h1>imageprocessing</h1>

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

# Create the train generator
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')

# Create the validation generator
validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical')


In [ ]:
# Retrieve the class indices
class_indices = train_generator.class_indices
print("Class indices:", class_indices)


<h1>cnn model architechrure</h1>

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(NUM_CLASSES, activation='softmax')
])


<h1>model training</h1>

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)



In [ ]:
# Additional epochs to train
additional_epochs = 30

# Continue training for additional epochs
history_additional = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=additional_epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE)

# Save the model in HDF5 format to the desktop directory
model.save(r'enter_the_path_to_save_the_model')

<h1>for predecting</h1>

In [ ]:


# Load the saved model
model = load_model(r"C:\Users\vikas\Desktop\garbage_model_test.keras")

# Define constants
IMAGE_SIZE = (150, 150)

# Define the image path
image_path = r"C:\Users\vikas\Downloads\garbage_classification\plastic\plastic765.jpg" # Replace 'path_to_your_test_image.jpg' with the actual path to your test image

# Define class labels mapping
class_labels = {
    0: 'Biological',
    1: 'Cardboard',
    2: 'Clothes',
    3: 'E-waste',
    4: 'Glass',
    5: 'Metal',
    6: 'Paper',
    7: 'Plastic',
    8: 'Shoes'
}

# Load and preprocess the image
img = image.load_img(image_path, target_size=IMAGE_SIZE)
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0  # Normalize the image data to [0, 1]

# Predict the class probabilities for the image
predictions = model.predict(img_array)

# Get the predicted class label
predicted_class = np.argmax(predictions)

# Get the corresponding class label
predicted_label = class_labels[predicted_class]

# Print the predicted class label
print("Predicted class:", predicted_label)

In [ ]:
train_acc = history.history['accuracy'] # store training accuracy in history
val_acc = history.history['val_accuracy'] # store validation accuracy in history
train_loss = history.history['loss'] # store training loss in history
val_loss = history.history['val_loss'] # store validation loss in history

epochs = range(1, len(train_acc) + 1)

plt.plot(epochs, train_acc, 'g*-', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, train_loss, 'g*-', label = 'Training loss')
plt.plot(epochs, val_loss, 'r', label = 'Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import accuracy_score



# Step 1: Predict classes for the dataset
y_pred = model.predict(X_dataset)  # Assuming X_dataset contains your image data

# Step 2: Calculate accuracy for each class
class_accuracies = []
for class_label in range(num_classes):
    # Filter predictions and ground truth for the current class
    y_true_class = y_dataset[y_dataset == class_label]
    y_pred_class = y_pred[y_dataset == class_label]
    
    # Calculate accuracy for the current class
    class_accuracy = accuracy_score(y_true_class, y_pred_class)
    class_accuracies.append(class_accuracy)

# Step 3: Print accuracy for each class
for class_label, accuracy in enumerate(class_accuracies):
    print(f"Class {class_label}: Accuracy = {accuracy:.2f}")

In [ ]:
plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
plt.bar(range(num_classes), class_accuracies)
plt.title('Class-wise Accuracy')
plt.xlabel('Class')
plt.ylabel('Accuracy')
plt.xticks(range(num_classes))  # Set x-axis ticks to class indices
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.show()